In [87]:
import os
import numpy as np
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import pandapower as pp
import pp_heig_simulation as pp_sim
import pp_heig_plot as pp_plot

création des bus


In [88]:
file_path = "data/trey_power_network.xlsx"  # ouverture OK
net_ExPF1_Pb1_heig = pp_sim.load_net_from_xlsx(file_path=file_path)

In [89]:
net_ExPF1_Pb1_heig

This pandapower network includes the following parameter tables:
   - bus (13 elements)
   - load (12 elements)
   - ext_grid (1 element)
   - line (11 elements)
   - trafo (1 element)

In [90]:
file_path2 = "data/liste_des_batiments_vf.csv"  # ouverture OK
df_habitations = pd.read_csv(file_path2, sep=",", encoding="latin-1")

In [91]:
df_habitations

,Affectation,Empreinte au sol [m2],Cabinet,geometry,egid,street,npa,city,canton,building_status,building_cath,building_class,building_year,building_month,building_area,floor_nb,room_nb,pc_sheltter,sre,heat_1,heat_source_1,heat_info_1,heat_date_1,heat_2,heat_source_2,heat_info_2,heat_date_2,heatwater_1,heatwater_source_1,heatwater_info_1,heatwater_date_1,heatwater_2,heatwater_source_2,heatwater_info_2,heatwater_date_2,edid,entrance_nb,esid,entrance_east,entrance_north,ewid,housing_area,housing_room_nb,housing_status,housing_year,kitchen
0,habitation et rural,569,CDBT016055,POINT (2560749.6 1179718.7),825586,Route de Payerne 30,1552,Trey,VD,built,mostly_residential,one_housing,1791.0,NaN,569,2.0,NaN,NaN,NaN,stove,Firewood,860.0,29.11.2001,NaN,NaN,NaN,-,NaN,Firewood,860,29.11.2001,NaN,unheated,860.0,29.11.2001,0,30,10146224,2560749.572,1179718.712,['1'],[270],[7],[3004],[1999],[1]
1,habitation,81,CDBT012139,POINT (2560730.2 1180098.9),825610,Impasse de la Forge 11,1552,Trey,VD,built,full_residential,one_housing,1991.0,NaN,81,3.0,NaN,NaN,NaN,boiler_1,gaz,869.0,07.04.2017,NaN,NaN,NaN,-,NaN,mazout,860,29.11.2001,NaN,unheated,860.0,29.11.2001,0,11,10153657,2560730.198,1180098.860,['1'],[140],[5],[3004],[1999],[1]
2,habitation et rural,147,CDBT004774,POINT (2560814.2 1179768.4),825588,Route de Payerne 21,1552,Trey,VD,built,full_residential,one_housing,NaN,NaN,147,2.0,NaN,NaN,136.0,PAC_1,unknown,869.0,19.01.2021,NaN,NaN,NaN,-,NaN,unknown,869,19.01.2021,NaN,NaN,NaN,-,0,21,10146224,2560814.213,1179768.411,['1'],[140],[6],[3004],[1999],[1]
3,habitation à affectation mixte,460,CDBT901452,POINT (2560802.3 1179957),825642,Route de Payerne 12b,1552,Trey,VD,built,mostly_residential,multi_housing,NaN,NaN,460,2.0,NaN,NaN,NaN,boiler_1,gaz,869.0,08.05.2014,NaN,NaN,NaN,-,NaN,gaz,869,08.05.2014,NaN,NaN,NaN,-,2,12b,10146224,2560802.325,1179957.035,['3' '4'],[190 190],[5 5],[3004 3004],[1999 1999],[1 1]
4,bâtiment agricole,252,STMT003438,POINT (2560881 1179966),280006187,Impasse du Coutset 1.1,1552,Trey,VD,built,no_residential,farm,NaN,NaN,252,NaN,NaN,0.0,NaN,NaN,NaN,NaN,-,NaN,NaN,NaN,-,NaN,unheated,869,30.11.2020,NaN,unheated,869.0,30.11.2020,1,1.1,10118880,2560881.000,1179966.000,NaN,NaN,NaN,NaN,NaN,NaN
5,habitation et rural,334,STMT003438,POINT (2560896.5 1179907.6),825600,Route du Coin 1,1552,Trey,VD,built,full_residential,one_housing,NaN,NaN,334,2.0,NaN,NaN,NaN,stove,mazout,860.0,29.11.2001,NaN,NaN,NaN,-,NaN,electricity,860,29.11.2001,NaN,unheated,860.0,29.11.2001,0,1,10141606,2560896.507,1179907.639,['1'],[84],[4],[3004],[1999],[1]
6,bâtiment agricole,1032,N1,POINT (2560582 1179898),280088599,Trey 1.2,1552,Trey,VD,built,no_residential,farm,2008.0,NaN,1032,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-,NaN,NaN,NaN,-,NaN,unheated,869,16.10.2014,NaN,unheated,869.0,16.10.2014,0,1.2,10041723,2560582.000,1179898.000,NaN,NaN,NaN,NaN,NaN,NaN
7,habitation,347,CDBT901604,POINT (2560753.9 1179632.4),825584,Chemin de Malafin 2,1552,Trey,VD,built,partially_residential,farm,NaN,NaN,347,2.0,1.0,NaN,NaN,NaN,mazout,NaN,-,NaN,NaN,NaN,-,NaN,mazout,860,29.11.2001,NaN,unheated,860.0,29.11.2001,0,2,10041722,2560753.922,1179632.357,['1'],[90],[3],[3004],[1999],[1]
8,habitation,84,CDBT012139,POINT (2560727.9 1180091.7),825612,Impasse de la Forge 13,1552,Trey,VD,built,full_residential,one_housing,1991.0,NaN,84,2.0,NaN,NaN,NaN,PAC_1,unknown,860.0,29.11.2001,NaN,NaN,NaN,-,NaN,mazout,860,29.11.2001,NaN,unheated,860.0,29.11.2001,0,13,10153657,2560727.850,1180091.672,['1'],[120],[5],[3004],[1999],[1]
9,habitation,303,CDBT004760,POINT (2560773.9 1180111.1),825614,Impasse de la Forge 4,1552,Trey,VD,built,full_residential,multi_housing,NaN,NaN,303,2.0,NaN,NaN,NaN,boiler_1,mazout,860.0,29.11.2001,NaN,NaN,NaN,-,NaN,Firewood,860,29.11.2001,NaN,NaN,NaN,-,1,4,10153657,2560773.900,1180111.116,['2'],[100],[4],[3004],[1999],[1]


In [92]:
df_habitations.loc[df_habitations["Cabinet"] == "STMT003438"]["Affectation"]

4       bâtiment agricole
5     habitation et rural
35    habitation et rural
37             habitation
38             habitation
Name: Affectation, dtype: object

In [93]:
##test = df_habitations.loc[df_habitations["Affectation"] == "habitation"]
df_habitations.loc[df_habitations["Affectation"] == "bâtiment industriel"]

,Affectation,Empreinte au sol [m2],Cabinet,geometry,egid,street,npa,city,canton,building_status,building_cath,building_class,building_year,building_month,building_area,floor_nb,room_nb,pc_sheltter,sre,heat_1,heat_source_1,heat_info_1,heat_date_1,heat_2,heat_source_2,heat_info_2,heat_date_2,heatwater_1,heatwater_source_1,heatwater_info_1,heatwater_date_1,heatwater_2,heatwater_source_2,heatwater_info_2,heatwater_date_2,edid,entrance_nb,esid,entrance_east,entrance_north,ewid,housing_area,housing_room_nb,housing_status,housing_year,kitchen
26,bâtiment industriel,37,CDBT004764,POINT (2560830 1180032),280025942,Place du Collège 1.1,1552,Trey,VD,built,no_residential,industial,1968.0,NaN,37,NaN,NaN,0.0,NaN,NaN,NaN,NaN,-,NaN,NaN,NaN,-,NaN,unheated,869,30.11.2020,NaN,unheated,869.0,30.11.2020,1,1.1,10141605,2560830.0,1180032.0,NaN,NaN,NaN,NaN,NaN,NaN


In [94]:
type_electrique = {
    "Affectation": [
        "habitation",
        "habitation et rural",
        "habitation à affectation mixte",
        "bâtiment agricole",
        "habitation et garage",
        "bâtiment ecclésiastique",
        "dépendance, bâtiment",
        "bâtiment industriel",
        "entrepôt",
        "garage collectif",
        "surface de dégagement",
        "Production (travail fin)",
        "Production (travail lourd)",
    ],
    "Pa": [10.0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13],
    "Pps": [0.0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13],
    "Pl": [7.7, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13],
    "Phi_c": [21.8, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13],
    "Phi:l": [19.0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13],
}
df_type_electrique = pd.DataFrame(type_electrique)
df_type_electrique

,Affectation,Pa,Pps,Pl,Phi_c,Phi:l
0,habitation,10.0,0.0,7.7,21.8,19.0
1,habitation et rural,2.0,2.0,2.0,2.0,2.0
2,habitation à affectation mixte,3.0,3.0,3.0,3.0,3.0
3,bâtiment agricole,4.0,4.0,4.0,4.0,4.0
4,habitation et garage,5.0,5.0,5.0,5.0,5.0
5,bâtiment ecclésiastique,6.0,6.0,6.0,6.0,6.0
6,"dépendance, bâtiment",7.0,7.0,7.0,7.0,7.0
7,bâtiment industriel,8.0,8.0,8.0,8.0,8.0
8,entrepôt,9.0,9.0,9.0,9.0,9.0
9,garage collectif,10.0,10.0,10.0,10.0,10.0


In [95]:
type_conso = {
    "Affectation": [
        "habitation",
        "entrepôt",
        "garage collectif",
        "surface de dégagement",
        "Production (travail fin)",
        "Production (travail lourd)",
    ],
    "Pa": [10.0, 2.0, 1.0, 0, 5.0, 5.0],
    "Pps": [0.0, 0, 0, 0, 10, 15],
    "Pl": [7.7, 7.3, 1.4, 3.5, 11.6, 7.0],
    "Phi_c": [21.8, 0, 0, 4, 32.2, 28.1],
    "Phi:l": [19.0, 18.8, 0, 6.9, 20.9, 18.8],
}
df_type_conso = pd.DataFrame(type_conso)